In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo to Show Anywidget mode

In [2]:
import bigframes.pandas as bpd

This notebook demonstrates the **anywidget** display mode for BigQuery DataFrames. This mode provides an interactive table experience for exploring your data directly within the notebook.

**Key features:**
- **Rich DataFrames & Series:** Both DataFrames and Series are displayed as interactive widgets.
- **Pagination:** Navigate through large datasets page by page without overwhelming the output.
- **Column Sorting:** Click column headers to toggle between ascending, descending, and unsorted views. Use **Shift + Click** to sort by multiple columns.
- **Column Resizing:** Drag the dividers between column headers to adjust their width.
- **Max Columns Control:** Limit the number of displayed columns to improve performance and readability for wide datasets.

In [3]:
bpd.options.bigquery.ordering_mode = "partial"
bpd.options.display.repr_mode = "anywidget"

Load Sample Data

## 1. Interactive DataFrame Display
Loading a dataset from BigQuery automatically renders the interactive widget.

In [4]:
df = bpd.read_gbq("bigquery-public-data.usa_names.usa_1910_2013")
print(df)

state gender  year     name  number
   AL      F  1910    Annie     482
   AL      F  1910   Myrtle     104
   AR      F  1910  Lillian      56
   CT      F  1910     Anne      38
   CT      F  1910  Frances      45
   FL      F  1910 Margaret      53
   GA      F  1910      Mae      73
   GA      F  1910 Beatrice      96
   GA      F  1910     Lola      47
   IA      F  1910    Viola      49
...

[5552452 rows x 5 columns]


In [ ]:
df

,state,gender,year,name,number
0,AL,F,1910,Hazel,51
1,AL,F,1910,Lucy,76
2,AR,F,1910,Nellie,39
3,AR,F,1910,Lena,40
4,CO,F,1910,Thelma,36
5,CO,F,1910,Ruth,68
6,CT,F,1910,Elizabeth,86
7,DC,F,1910,Mary,80
8,FL,F,1910,Annie,101
9,FL,F,1910,Alma,39


## 2. Interactive Series Display
BigQuery DataFrames `Series` objects now also support the full interactive widget experience, including pagination and formatting.

In [6]:
test_series = df["year"]
# Displaying the series triggers the interactive widget
print(test_series)

1910
1910
1910
1910
1910
1910
1910
1910
1910
1910
Name: year, dtype: Int64
...

[5552452 rows]


Display with Pagination

In [7]:
test_series

1910
1910
1910
1910
1910
1910
1910
1910
1910
1910
Name: year, dtype: Int64
...

[5552452 rows]

### Sorting by Column(s)
You can sort the table by clicking on the headers of columns that have orderable data types (like numbers, strings, and dates). Non-orderable columns (like arrays or structs) do not have sorting controls.

#### Single-Column Sorting
The sorting control cycles through three states:
- **Unsorted (no indicator by default, ● on hover):** The default state. Click the header to sort in ascending order.
- **Ascending (▲):** The data is sorted from smallest to largest. Click again to sort in descending order.
- **Descending (▼):** The data is sorted from largest to smallest. Click again to return to the unsorted state.

#### Multi-Column Sorting
You can sort by multiple columns to further refine your view:
- **Shift + Click:** Hold the `Shift` key while clicking additional column headers to add them to the sort order. 
- Each column in a multi-sort also cycles through the three states (Ascending, Descending, Unsorted).
- **Indicator visibility:** Sorting indicators (▲, ▼) are always visible for all columns currently included in the sort. The unsorted indicator (●) is only visible when you hover over an unsorted column header.

### Adjustable Column Widths
You can easily adjust the width of any column in the table. Simply hover your mouse over the vertical dividers between column headers. When the cursor changes to a resize icon, click and drag to expand or shrink the column to your desired width. This allows for better readability and customization of your table view.

### Control Maximum Columns
You can control the number of columns displayed in the widget using the **Max columns** dropdown in the footer. This is useful for wide DataFrames where you want to focus on a subset of columns or improve rendering performance. Options include 3, 5, 7, 10, 20, or All.

Programmatic Navigation Demo

## 3. Programmatic Widget Control
You can also instantiate the `TableWidget` directly for more control, such as checking page counts or driving navigation programmatically.

In [8]:
from bigframes.display.anywidget import TableWidget
import math
  
# Create widget programmatically  
widget = TableWidget(df)
print(f"Total pages: {math.ceil(widget.row_count / widget.page_size)}")
  
# Display the widget
widget

Total pages: 555246


Test Navigation Programmatically

In [9]:
# Simulate button clicks programmatically
print("Current page:", widget.page)

# Go to next page
widget.page = 1
print("After next:", widget.page)

# Go to previous page
widget.page = 0
print("After prev:", widget.page)

Current page: 0
After next: 1
After prev: 0


## 4. Edge Cases
The widget handles small datasets gracefully, disabling unnecessary pagination controls.

In [10]:
# Test with very small dataset
small_df = df.sort_values(["name", "year", "state"]).head(5)
small_widget = TableWidget(small_df)
print(f"Small dataset pages: {math.ceil(small_widget.row_count / small_widget.page_size)}")
small_widget

Small dataset pages: 1


### Displaying Generative AI results containing JSON
The `AI.GENERATE` function in BigQuery returns results in a JSON column. While BigQuery's JSON type is not natively supported by the underlying Arrow `to_pandas_batches()` method used in anywidget mode ([Apache Arrow issue #45262](https://github.com/apache/arrow/issues/45262)), BigQuery Dataframes automatically converts JSON columns to strings for display. This allows you to view the results of generative AI functions seamlessly.

## 5. Advanced Data Types (JSON/Structs)
The `AI.GENERATE` function in BigQuery returns results in a JSON column. BigQuery Dataframes automatically handles complex types like JSON strings for display, allowing you to view generative AI results seamlessly.

In [11]:
bpd._read_gbq_colab("""
  SELECT
    AI.GENERATE(
      prompt=>(\"Extract the values.\", OBJ.GET_ACCESS_URL(OBJ.FETCH_METADATA(OBJ.MAKE_REF(gcs_path, \"us.conn\")), \"r\")),
      connection_id=>\"bigframes-dev.us.bigframes-default-connection\",
      output_schema=>\"publication_date string, class_international string, application_number string, filing_date string\") AS result,
    *
  FROM `bigquery-public-data.labeled_patents.extracted_data`
  LIMIT 5;
""")

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)


/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)
/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)


,result,gcs_path,issuer,language,publication_date,class_international,class_us,application_number,filing_date,priority_date_eu,representative_line_1_eu,applicant_line_1,inventor_line_1,title_line_1,number
0,"{'application_number': None, 'class_internatio...",gs://gcs-public-data--labeled-patents/espacene...,EU,DE,03.10.2018,H01L 21/20,<NA>,18166536.5,16.02.2016,<NA>,"Scheider, Sascha et al",EV Group E. Thallner GmbH,"Kurz, Florian",VORRICHTUNG ZUM BONDEN VON SUBSTRATEN,EP 3 382 744 A1
1,"{'application_number': None, 'class_internatio...",gs://gcs-public-data--labeled-patents/espacene...,EU,DE,03.10.2018,A01K 31/00,<NA>,18171005.4,05.02.2015,05.02.2014,Stork Bamberger Patentanw√§lte,Linco Food Systems A/S,"Thrane, Uffe",MASTH√ÑHNCHENCONTAINER ALS BESTANDTEIL EINER E...,EP 3 381 276 A1
2,"{'application_number': None, 'class_internatio...",gs://gcs-public-data--labeled-patents/espacene...,EU,DE,03.10.2018,G06F 11/30,<NA>,18157347.8,19.02.2018,31.03.2017,Hoffmann Eitle,FUJITSU LIMITED,"Kukihara, Kensuke","METHOD EXECUTED BY A COMPUTER, INFORMATION PRO...",EP 3 382 553 A1
3,"{'application_number': None, 'class_internatio...",gs://gcs-public-data--labeled-patents/espacene...,EU,DE,03.10.2018,H05B 6/12,<NA>,18165514.3,03.04.2018,30.03.2017,<NA>,BSH Hausger√§te GmbH,"Acero Acero, Jesus",VORRICHTUNG ZUR INDUKTIVEN ENERGIE√úBERTRAGUNG,EP 3 383 141 A2
4,"{'application_number': None, 'class_internatio...",gs://gcs-public-data--labeled-patents/espacene...,EU,DE,29.08.018,E04H 6/12,<NA>,18157874.1,21.02.2018,22.02.2017,Liedtke & Partner Patentanw√§lte,SHB Hebezeugbau GmbH,"VOLGER, Alexander",STEUERUNGSSYSTEM F√úR AUTOMATISCHE PARKH√ÑUSER,EP 3 366 869 A1
